In [ ]:
%run NB_Credentials

StatementMeta(, 5d75dff1-333f-4f2a-b9c0-dcfcedd3d282, 11, Finished, Available, Finished)

In [ ]:
import requests
import pandas as pd
import time
from pyspark.sql.functions import col, max as spark_max

StatementMeta(, 5d75dff1-333f-4f2a-b9c0-dcfcedd3d282, 12, Finished, Available, Finished)

In [ ]:
# Iniciar tiempo de ejecución
start_time = time.time()

StatementMeta(, 5d75dff1-333f-4f2a-b9c0-dcfcedd3d282, 13, Finished, Available, Finished)

In [ ]:
# Parámetros
SEARCH_QUERIES = ["PowerBI", "MicrosoftFabric", "Qlik", "Tableau"]  
# SEARCH_QUERIES = "powerbi"
NUM_POSTS = 50
LIMIT_PER_REQUEST = 100

# Ruta del Data Lake y tabla de destino
TABLA_DESTINO = "bronze_fact_post"

StatementMeta(, 5d75dff1-333f-4f2a-b9c0-dcfcedd3d282, 14, Finished, Available, Finished)

In [ ]:
# Lista para almacenar los posts
posts_data = []

# Obtener la última marca de tiempo por cada búsqueda desde el Lakehouse
latest_timestamps = {}
try:
    existing_df = spark.read.format("delta").load(f"{BRONZE_PATH}/{TABLA_DESTINO}")
    for query in SEARCH_QUERIES:
        max_date = existing_df.filter(col("search_query") == query).agg(spark_max("date")).collect()[0][0]
        latest_timestamps[query] = max_date if max_date else "1970-01-01T00:00:00Z"
    print(f"🕒 Últimas marcas de tiempo por búsqueda: {latest_timestamps}")
except Exception as e:
    print("⚠️ No se encontraron datos existentes o error al leer el Lakehouse. Iniciando desde cero.")
    for query in SEARCH_QUERIES:
        latest_timestamps[query] = "1970-01-01T00:00:00Z"

StatementMeta(, 5d75dff1-333f-4f2a-b9c0-dcfcedd3d282, 15, Finished, Available, Finished)

🕒 Últimas marcas de tiempo por búsqueda: {'PowerBI': '2025-02-20T16:13:27.652Z', 'MicrosoftFabric': '2025-02-20T15:16:06.743Z', 'Qlik': '2025-02-18T10:06:01.164Z', 'Tableau': '2025-02-20T16:11:25.214Z'}


In [ ]:
# Obtener posts para cada término de búsqueda
for query in SEARCH_QUERIES:
    # print(f"\n🔍 Buscando posts para: {query}")
    
    cursor = None
    posts_collected = 0  # Contador de posts por búsqueda
    latest_saved = latest_timestamps[query]

    while posts_collected < NUM_POSTS:
        params = {"q": query, "limit": LIMIT_PER_REQUEST}
        if cursor:
            params["cursor"] = cursor  # Usar cursor si está disponible

        response = requests.get("https://public.api.bsky.app/xrpc/app.bsky.feed.searchPosts", params=params)

        if response.status_code != 200:
            print(f"❌ Error en la petición: {response.status_code}")
            print("Mensaje de error:", response.text)
            break  # Detener en caso de error

        data = response.json()
        posts = data.get("posts", [])
        new_cursor = data.get("cursor")  # Obtener nuevo cursor

        if not posts:
            print("🚫 No se encontraron más posts.")
            break  # Detener si no hay más posts

        for post in posts:
            created_at = post.get("record", {}).get("createdAt", "")

            # Lógica incremental: solo posts más recientes
            if created_at > latest_saved:
                posts_data.append({
                    "date": created_at,
                    "author": post.get("author", {}).get("handle", ""),
                    "post": post.get("record", {}).get("text", ""),
                    "search_query": query
                })

        posts_collected += len(posts)

        # Si el cursor no cambia, detener la paginación
        if not new_cursor or new_cursor == cursor:
            break

        cursor = new_cursor  # Actualizar cursor
        time.sleep(2)  # Pausa para evitar bloqueos

    # print(f"✅ Recibidos {posts_collected} posts para '{query}'.")

StatementMeta(, 5d75dff1-333f-4f2a-b9c0-dcfcedd3d282, 16, Finished, Available, Finished)

In [ ]:
# Convertir a DataFrame y guardar en el Lakehouse
if posts_data:
    df_posts = pd.DataFrame(posts_data)
    print(f"📊 Total de nuevos posts recopilados: {len(df_posts)}")

    # Guardar en formato Delta
    spark.createDataFrame(df_posts).write \
        .format("delta") \
        .mode("append") \
        .option("mergeSchema", "true") \
        .partitionBy("date") \
        .save(f"{BRONZE_PATH}/{TABLA_DESTINO}")
else:
    print("🚫 No hay nuevos posts para guardar.")

StatementMeta(, 5d75dff1-333f-4f2a-b9c0-dcfcedd3d282, 17, Finished, Available, Finished)

🚫 No hay nuevos posts para guardar.


In [ ]:
# Calcular tiempo de ejecución
end_time = time.time()
execution_time = end_time - start_time
print(f"⏳ Tiempo de ejecución: {execution_time:.2f} segundos")

StatementMeta(, 5d75dff1-333f-4f2a-b9c0-dcfcedd3d282, 18, Finished, Available, Finished)

⏳ Tiempo de ejecución: 18.82 segundos
